## 6-3　BERT による文章分類

In [1]:
# !mkdir chap6
%cd ./chap6

/content/chap6


In [2]:
# !pip install transformers==4.18.0 fugashi===1.1.0 ipadic==1.0.0 pytorch-lightning==1.6.1

In [3]:
import glob
import torch
import random
import numpy as np
import pytorch_lightning as pl

from tqdm import tqdm
from torch.utils.data import DataLoader
from transformers import BertJapaneseTokenizer, BertForSequenceClassification

In [4]:
MODEL_NAME = 'tohoku-nlp/bert-base-japanese-whole-word-masking'

In [5]:
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)
bert_sc = BertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)
bert_sc = bert_sc.cuda()

Some weights of the model checkpoint at tohoku-nlp/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initializ

In [6]:
text_list = [
    'この映画は面白かった。',
    'この映画の最後にはがっかりさせられた。',
    'この映画を見て幸せな気持ちになった。'
    ]

label_list = [1, 0, 1]

encoding = tokenizer(text_list, padding='longest', return_tensors='pt')
encoding = {k: v.cuda() for k, v in encoding.items()}
labels = torch.tensor(label_list).cuda()

with torch.no_grad():
  output = bert_sc.forward(**encoding)
score = output.logits
labels_predicted = score.argmax(-1)
num_correct = (labels_predicted==labels).sum().item()
accuracy = num_correct / labels.size(0)

print('# scores:')
print(score.size())
print('# predicted labels:')
print(labels_predicted)
print('# accuracy')
print(accuracy)

# scores:
torch.Size([3, 2])
# predicted labels:
tensor([1, 1, 1], device='cuda:0')
# accuracy
0.6666666666666666


In [7]:
encoding = tokenizer(text_list, padding='longest', return_tensors='pt')
encoding['labels'] = torch.tensor(label_list)
encoding = {k: v.cuda() for k, v in encoding.items()}

output = bert_sc(**encoding)
loss = output.loss
print(loss)

tensor(0.7040, device='cuda:0', grad_fn=<NllLossBackward0>)
